# Notes

## Loading the Code

Comet_ML library and experiment must be run before mlflow and sklearn. They cannot be run in the same cell if you are using jupyter notebook 

## Limitations of Comet

Note: 
Comet does not support 
* Does not support MLFlow nested runs.
* Does not support continuing a previous MLFlow run. The MLFlow extension will create a new Comet Experiment in that case.


### Does not Support a Previous MLflow Run
Ml flow usually contains multiple experiments in a run. However Comet's experiment tracking tool creates an experiment for each time the code is run. This can get very confusing, and should be noted or tagged for each experiments or in a notes. 

### Does not Support MLflow nested runs. 
This means that you cannot use the with statement to run the MLflow code. It must be outside of the with statement. Mlflow runs that are enhanced using Comet need to be denested.

# Code

In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np


# Import comet_ml at the top of your file - these must be imported before sklearn or mlflow
from comet_ml import Experiment
import comet_ml

In [2]:

import mlflow
import mlflow.sklearn

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet


In [3]:
# Create an experiment with your api key
experiment = Experiment(
    api_key="xCeYnrykwJF1pzF0Rfj8UEzR2",
    project_name="comet_mlflow", 
    workspace="mattblasa",
    log_code = True
)


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mattblasa/comet-mlflow/a83dbe6663c6401581f7054e644124d2



In [4]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


In [5]:
warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the wine-quality csv file (make sure you're running this from the root of MLflow!)

data = pd.read_csv('wine-quality.csv')

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

#alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.5
#l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.5

alpha = 0 
l1_ratio = 0

with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/mattblasa/comet-mlflow/a83dbe6663c6401581f7054e644124d2
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mattblasa/general/a17e0b4fb41b41e6beb6e825abe72c48



Elasticnet model (alpha=0.000000, l1_ratio=0.000000):
  RMSE: 0.7424162938559533
  MAE: 0.5775168907128393
  R2: 0.2881067715835024


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/mattblasa/general/a17e0b4fb41b41e6beb6e825abe72c48
COMET INFO:   Metrics:
COMET INFO:     mae  : 0.5775168907128393
COMET INFO:     r2   : 0.2881067715835024
COMET INFO:     rmse : 0.7424162938559533
COMET INFO:   Others:
COMET INFO:     Created from : MLFlow auto-logger
COMET INFO:   Parameters:
COMET INFO:     alpha    : 0
COMET INFO:     l1_ratio : 0
COMET INFO:   Uploads:
COMET INFO:     asset               : 4 (1.29 KB)
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (m

# Model Building

In [ ]:
mlflow.log_metric("score", score)
mlflow.sklearn.log_model(lr, "model")

In [ ]:
print("Model saved in run %s" % mlflow.active_run().info.run_uuid)